In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [2]:
from loader import load_process_split_doc, load_process_split_doc_law

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file))
# load - 참고자료
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))
    

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3

In [6]:
len(all_documents)

6241

In [ ]:
# 📌 vector store 생성
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
vector_store = set_vector_store(all_documents)

In [ ]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
)

In [ ]:
# vector_store._collection.count()

In [1]:
# Retreiver 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 5, "lambda_mult":1}
)

In [ ]:
# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 

    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온다.
        - 예를들어 '개별소비세법 1조의 내용'이라는 질문을 받으면 개별소비세법 1조의 조항을 전부 다 답변한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계하며, 각 질문 앞뒤에 한 줄씩 띄어쓰기를 합니다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."

    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [ ]:
print(chain.invoke("법인세법 제1조 에 대해 알려줘"))
retriever.invoke("법인세법 제1조 에 대해 알려줘")

법인세법 제1조는 법인세법의 목적에 대해 규정하고 있습니다. 일반적으로 법 제1조는 해당 법률의 목적과 기본적인 방향성을 설명합니다. 법인세법 제1조는 법인세의 부과 및 징수에 관한 사항을 규정함으로써 국가 재정에 기여하고, 법인의 경제 활동을 적절히 통제하기 위한 기본적인 목적을 명시하고 있을 것입니다.

자세한 조문 내용은 법인세법 원문을 참고하시기 바랍니다. 법률 문서에서 제1조는 대개 법의 목적을 설명하는 조항입니다. 추가적인 세부 사항이나 특정 질문이 있으시면 말씀해 주세요.


[Document(metadata={'description': '법인세법 시행규칙 에 관한 문서입니다.', 'keyword': '법인세법_시행규칙', 'source': '법인세법_시행규칙.pdf'}, page_content='법인세법 시행규칙\n \n기획재정부 (국제조세제도과(외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 시행령 에 관한 문서입니다.', 'keyword': '법인세법_시행령', 'source': '법인세법_시행령.pdf'}, page_content='법인세법 시행령\n \n기획재정부 (국제조세제도과(외국법인)) 044-215-4652\n기획재정부 (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 에 관한 문서입니다.', 'keyword': '법인세법', 'source': '법인세법.pdf'}, page_content='법인세법\n \n기획재정부 (법인세제과) 044-215-4221\n기획재정부 (국제조세제도과(외국법인)) 044-215-4652\n       제1장 총칙')]

## 평가

In [ ]:
# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비
test_data = [
    # {
    #     "question": "개별소비세법의 목적은 무엇인가요?",
    #     "answer": chain.invoke("개별소비세법의 목적은 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")],
    #     "ground_truths": ["개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")])
    # },
    {
        "question": "개별소비세법 제1조가 무엇인가요?",
        "answer": chain.invoke("개별소비세법 제1조가 무엇인가요?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")],
        "ground_truths": ["개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다."],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")])
    },
    # {
    #     "question": "조세범 처벌절차법의 목적이 무엇인가요?",
    #     "answer": chain.invoke("조세범 처벌절차법의 목적이 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")],
    #     "ground_truths": ["제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")])
    # },
    
]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'faithfulness': 0.3000, 'answer_relevancy': 0.8789, 'context_precision': 1.0000, 'context_recall': 1.0000}


In [2]:
# GPT-4o 모델을 사용하여 평가 
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
langchain_model = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextRecall(llm=langchain_model),
    LLMContextPrecisionWithReference(llm=langchain_model),
    Faithfulness(llm=langchain_model),
    AnswerRelevancy(llm=langchain_model, embeddings=eval_embedding)
]
result = evaluate(dataset=dataset, metrics=metrics)
result

In [ ]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)
# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)
# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result

,question,answer,contexts,ground_truths,reference
0,개별소비세법의 목적은 무엇인가요?,"개별소비세법의 목적은 특정 물품이나 서비스에 대해 소비세를 부과하여 세수 확보와 소비 패턴 조정을 통해 공공재원을 조달하고 사회적 비용을 반영하는 것입니다. 이를 통해 사치품이나 환경에 미치는 영향이 큰 제품의 소비를 억제하고, 공공 목적으로 사용되는 자금을 확보하는 데 기여합니다. \n\n추가적으로 궁금하신 사항이 있나요?\n\n1. 개별소비세가 부과되는 구체적인 품목에는 어떤 것들이 있는지 알고 싶으신가요?\n \n2. 개별소비세의 세율은 어떻게 결정되는지 궁금하신가요?","[개별소비세법 시행규칙\n[시행 2024. 3. 22.] [기획재정부령 제1047호, 2024. 3. 22., 일부개정]\n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336, 제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. <개정 2015. 12. 15.>\n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n[전문개정 2010. 1. 1.], 제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다., 제1조(목적) 이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2010. 4. 13.], 제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의\n공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. <개정\n2020. 6. 9.>\n[전문개정 2010. 1. 1.]]",[개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다.],"개별소비세법 시행규칙\n[시행 2024. 3. 22.] [기획재정부령 제1047호, 2024. 3. 22., 일부개정]\n기획재정부 (환경에너지세제과) 044-215-4331, 4336\n기획재정부 (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336\n제4조(과세시기) 개별소비세는 다음 각 호에 따른 반출, 수입신고, 입장, 유흥음식행위 또는 영업행위를 할 때에 그 행위\n당시의 법령에 따라 부과한다. 다만, 제3조제4호의 경우에는 「관세법」에 따른다. <개정 2015. 12. 15.>\n1. 물품에 대한 개별소비세: 과세물품을 제조장에서 반출할 때 또는 수입신고를 할 때\n2. 입장행위에 대한 개별소비세: 과세장소에 입장할 때\n3. 유흥음식행위에 대한 개별소비세: 유흥음식행위를 할 때\n 4. 영업행위에 대한 개별소비세: 과세영업장소의 영업행위를 할 때\n[전문개정 2010. 1. 1.]\n제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n제1조(목적) 이 규칙은 「외국인관광객 등에 대한 부가가치세 및 개별소비세 특례규정」에서 위임된 사항과 그 시행에\n필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2010. 4. 13.]\n제1조(목적) 이 법은 조세(租稅)의 감면 또는 중과(重課) 등 조세특례와 이의 제한에 관한 사항을 규정하여 과세(課稅)의\n공평을 도모하고 조세정책을 효율적으로 수행함으로써 국민경제의 건전한 발전에 이바지함을 목적으로 한다. <개정\n2020. 6. 9.>\n[전문개정 2010. 1. 1.]"
1,조세범 처벌절차법의 목적이 무엇인가요?,조세범 처벌절차법의 목적은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납세 의식을 확립하는 데 있습니다. 이 법은 세법 집행의 공정성과 세금 납부의 적정성을 보장하기 위한 제도적 장치를 마련하고 있습니다.\n\n세법과 관련된 다른 질문이 있으신가요? 예를 들어:\n\n1. 조세범 처벌절차법에 의해 처벌받는 구체적인 행위는 어떤 것이 있는지 알고 싶으신가요?\n\n2. 조세범 처벌절차법과 관련된 최신 개정 사항이 궁금하신가요?,"[조세범 처벌절차법\n[시행 2023. 1. 17.] [법률 제19212호, 2023. 1. 17., 일부개정]\n기획재정부 (조세법령운용팀) 044-215-4151\n 제1장 총칙, 제1조(목적) 이 영은 「조세범 처벌절차법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다., 제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다., 제1조(목적) 이 법은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납\n세의식을 확립함을 목적으로 한다. <개정 2018. 12. 31.>, 제39조(벌칙) 재평가세에 관한 범칙행위에 대하여는 조세범처벌법을 적용한다.]",[제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다],"조세범 처벌절차법\n[시행 2023. 1. 17.] [법률 제19212호, 2023. 1. 17., 일부개정]\n기획재정부 (조세법령운용팀) 044-215-4151\n 제1장 총칙\n제1조(목적) 이 영은 「조세범 처벌절차법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n제1조(목적) 이 법은 주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을\n확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다.\n제1조(목적) 이 법은 세법을 위반한 자에 대한 형벌에 관한 사항을 규정하여 세법의 실효성을 높이고 국민의 건전한 납\n세의식을 확립함을 목적으로 한다. <개정 2018. 12. 31.>\n제39조(벌칙) 재평가세에 관한 범칙행위에 대하여는 조세범처벌법을 적용한다."
